We will start by importing some modules and loading in our json data. Make sure that your .json file is in the same directory as your python script!

In [8]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import numpy
import tflearn
import tensorflow
import json

with open('intents.json') as file:
    data = json.load(file)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/satriahidayat/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [9]:
words = []
labels = []
docs_x = []
docs_y = []

Now its time to loop through our JSON data and extract the data we want. For each pattern we will turn it into a list of words using nltk.word_tokenizer, rather than having them as strings. We will then add each pattern into our docs_x list and its associated tag into the docs_y list.

In [10]:
for intent in data['intents']:
    for pattern in intent['patterns']:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])
        
    if intent['tag'] not in labels:
        labels.append(intent['tag'])

This code will simply create a unique list of stemmed words to use in the next step of our data preprocessing.

In [11]:
words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

labels = sorted(labels)

Now that we have loaded in our data and created a stemmed vocabulary it's time to talk about a bag of words

In [12]:
training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []

    wrds = [stemmer.stem(w.lower()) for w in doc]

    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)

Finally we will convert our training data and output to numpy arrays.

In [13]:
training = numpy.array(training)
output = numpy.array(output)

We will start by defining the architecture of our model

In [14]:
tensorflow.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

Now that we have setup our model its time to train it on our data! To do these we will fit our data to the model. The number of epochs we set is the amount of times that the model will see the same information while training.

In [15]:
model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
model.save("model.tflearn")

Training Step: 4999  | total loss: 0.23733 | time: 0.028s
| Adam | epoch: 1000 | loss: 0.23733 - acc: 0.9653 -- iter: 32/40
Training Step: 5000  | total loss: 0.21964 | time: 0.035s
| Adam | epoch: 1000 | loss: 0.21964 - acc: 0.9688 -- iter: 40/40
--
INFO:tensorflow:/Users/satriahidayat/Desktop/ML/BOT/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [ ]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return numpy.array(bag)

The bag_of_words function will transform our string input to a bag of words using our created words list.

In [ ]:
def chat():
    print("Start talking with the bot (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        results = model.predict([bag_of_words(inp, words)])
        results_index = numpy.argmax(results)
        tag = labels[results_index]
        print("intent: " + tag)

chat()